# Processing - Fit and get features from flash response
Fit flash responde to probability distribution function and create a PSTH of response for each unit in spiketime file. Classificate the light response in **null, on, off and onoff** response and get the following features:
- latency (on and off)
- decay time (on and off)
- bias index
- max response (on and off)
- response index (on and off)
- sustained index (on and off)

Some reference https://www.ncbi.nlm.nih.gov/pubmed/12966177

In [1]:
from configparser import ConfigParser, ExtendedInterpolation

import h5py
import numpy as np
import pandas as pd

from spikelib.utils import check_directory

from flash import get_flash_features, get_flash_response
%matplotlib inline

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

['../../config.ini']

# Processing - Flash features


## General parameters

In [2]:
exp_name = config['EXP']['name']
sync_file = config['SYNC']['events']
sorting_file = config['FILES']['sorting']
output_file = config['FILES']['processed']
print('OUTPUT FILE: ' + output_file)

# Details about flash parameter
protocol_name = 'flash'
# nframes = 24 # Number of frame for ON stimulus. If fps=60, 24 frames = 0.4s

# Temporal resolution
sample_rate = 20000.0
psth_bin = 0.01  # In sec
bandwidth_fit = psth_bin
fit_resolution = 0.001  # In sec

OUTPUT FILE: /home/vnc/Downloads/MEA-analysis-master/data/processed_protocols/MR-0061__analysis_of_protocols.hdf5


In [3]:
# Read protocols times from sync file
protocol_times = pd.read_csv(sync_file, sep=',')
protocol_times['extra_desciption'].fillna('', inplace=True)

flashe_times = protocol_times[protocol_times['protocol_name'] == protocol_name]
protocol_flashes = flashe_times[['protocol_name', 'nd', 'intensity', 'extra_desciption']].drop_duplicates().values
protocol_flashes

array([['flash', 'nd3', 50.0, 'cyan'],
       ['flash', 'nd3', 100.0, 'cyan'],
       ['flash', 'nd3', 150.0, 'cyan'],
       ['flash', 'nd3', 200.0, 'cyan'],
       ['flash', 'nd3', 255.0, 'cyan']], dtype=object)

## Get PSTH and estimated firing rate

In [4]:
for (protocol_name, nd, intensity, extra) in protocol_flashes:
    with h5py.File(sorting_file, 'r') as spks,\
         h5py.File(output_file, 'a') as panalysis:
        get_flash_response(
            spks=spks,
            panalysis=panalysis,
            protocol_name=protocol_name,
            nd=nd,
            intensity=intensity,
            extra=extra,
            event_list=protocol_times,
            psth_bin=psth_bin,
            bandwidth_fit=bandwidth_fit,
            fit_resolution=fit_resolution,
            sr=20000.0
        )


## Compute flash features

In [5]:
#keywords for spkt.get_features_flash()
fpeak_min_time = 0.01  # min time between peaks in sec
kwargs_fit = {
    'resp_thr': 1.0/3,  # Threshold to select valid unit based on number of trials
    'bias_thr': 0.65,  # Threshold to classify into on, off, onoff
    'ri_thr': 0.3,  # Threshold for Response Index (RI)
    'ri_span': 0.1,  # Span for Response Index (RI)
    'fpeak_thr': 0.5,  # threshold of max response to find the first peak of response
    'fpeak_min_dist': int(fpeak_min_time/fit_resolution),
    'sust_time': 0.4,  # Windows time to compute Sustained index in seg
    'decrease_factor': np.e,
}

for (protocol_name, nd, intensity, extra) in protocol_flashes:
    with h5py.File(output_file, 'a') as panalysis:
        get_flash_features(
            panalysis=panalysis,
            protocol_name=protocol_name,
            nd=nd, 
            intensity=intensity, 
            extra=extra,
            kwargs_fit=kwargs_fit
        )

In [6]:
with h5py.File(output_file, 'r') as panalysis:
    print(panalysis['flash/nd3/'].keys())

<KeysViewHDF5 ['050_cyan', '100_cyan', '150_cyan', '200_cyan', '255_cyan']>
